In [1]:
import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import string
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%load_ext autoreload
%autoreload 2

from rnn import RNN, PrepareData

In [2]:
dataset = PrepareData('input.txt', batch_size=30, val_ratio=0.2)
train_data, val_data = dataset.dataset()

In [3]:
print(len(train_data), len(val_data))
print(train_data[0], val_data[0])

13073 3268
 dc | d2 Bd |
B2d2 | B3B | GB/  B2G|
~D3 D2A|A2G E2G|1 ABA G2


In [5]:
#input_dim = 7
hidden_dim = 20

all_letters = string.printable
n_letters = len(all_letters)

model = RNN(n_letters, hidden_dim, n_letters)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

In [6]:
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for idx, letter in enumerate(line[:-1]):
        tensor[idx][0][all_letters.find(letter)] = 1
    return Variable(tensor.cuda())
        
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in line[1:]]
    return Variable(torch.LongTensor(letter_indexes).cuda())

In [ ]:
n_epochs = 10
all_losses = []
print_every = 2000

for epoch in range(n_epochs):
    train_cnt = 0
    total_loss = 0.
    
    dataset.shuffle_data()
    train_data, val_data = dataset.dataset()
    
    for sentence in train_data:
        model.zero_grad()
        model.hidden = model.init_hidden()
        
        loss = 0.
        inputs = inputTensor(sentence)
        targets = targetTensor(sentence)
        
        for i in range(len(sentence)-1):
            preds = model(inputs[i])
            loss += criterion(preds, targets[i])
            
        loss.backward()
        optimizer.step()
        total_loss += loss
        train_cnt += 1
        
        if train_cnt % print_every == 0:
            total_loss /= print_every
            all_losses.append(total_loss)
            print('[%d, %d], loss: %f' % (epoch, train_cnt, total_loss))
            total_loss = 0

[0, 2000], loss: 131.791107
[0, 4000], loss: 130.716431
[0, 6000], loss: 129.476166
[0, 8000], loss: 127.689384
[0, 10000], loss: 125.668579
[0, 12000], loss: 123.161575
[1, 2000], loss: 119.516609
[1, 4000], loss: 117.449081
[1, 6000], loss: 115.565659
[1, 8000], loss: 114.020439
[1, 10000], loss: 112.319237
[1, 12000], loss: 111.324867
[2, 2000], loss: 109.563766
[2, 4000], loss: 108.695984
[2, 6000], loss: 108.117638
[2, 8000], loss: 107.459076
[2, 10000], loss: 107.260818
[2, 12000], loss: 107.083755
[3, 2000], loss: 106.395088


In [ ]:
max_length = 20

test_sen = val_data[0]
inputs = inputTensor(test_sen)
for i in range(len(test_sen)-1):
    output = model(inputs[i])
    _, pred = torch.max(output.data, 1)
    print(all_letters[int(pred)])
print(test_sen)

In [ ]:
plt.figure()
plt.plot(all_losses)